In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
import pandas as pd
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import numpy as np
import filter_dataset
from pyvis import network as net 


user_name = input("Enter user name: ")
# df = pd.read_csv("/random_data.csv")
# df1 = pd.read_csv("profile_dataNewest.csv")
df1 = pd.read_csv("R:/Git/Relator_Bot/test datasets/1000_location.csv")
df = filter_dataset.filter_ds(user_name, df1)
torch.manual_seed(34)

def gcn_main():
    G = nx.Graph()
    label = {}
    likes = {}
    search = {}  # Initialize the search dictionary

    user_data = df[['first_name', 'categories']].values
    user_names = df['first_name'].tolist()
    user_count = len(user_names)

    for i, (user, category) in enumerate(user_data):
        label[i] = user
        search[user] = i  # Map user names to indices
        G.add_node(i, title=user, label=user)
        likes[user] = set(category.split(','))

    edges = []
    for u in range(user_count):
        for v in range(u + 1, user_count):
            common_categories = likes[user_names[u]].intersection(likes[user_names[v]])
            if common_categories:
                union_categories = likes[user_names[u]].union(likes[user_names[v]])
                similarity_score = float(len(common_categories) / len(union_categories)) * 100
                edges.append((u, v, similarity_score))

    if not edges:
        print("Error: The graph has no edges.")
        return

    for u, v, w in edges:
        G.add_edge(u, v, weight=w)

    edge_index = torch.tensor([(u, v) for u, v, _ in edges]).t().contiguous()
    x = torch.randn(user_count, 16)
    data = Data(x=x, edge_index=edge_index)

    gcn_model = GCNModel(input_dim=16, hidden_dim=32, output_dim=1)
    gcn_output = gcn_model(data)

    visualize_graph(G, label, search, gcn_output, likes)

class GCNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        return x

def visualize_graph(G, label, search, gcn_output, likes):
    user_index = search.get(user_name)
    print(user_index)
    if user_index is not None:
        get_score(G, label, user_index, gcn_output, likes)
    else:
        print("User not found.")

    nt = net.Network(notebook=True, cdn_resources='remote', height="900px", width="100%", bgcolor="#222222", font_color="white")
    nt.from_nx(G, default_node_size=100, show_edge_weights=True, edge_scaling=True)
    neighbor_map = nt.get_adj_list()

    for node in nt.nodes:
        neighbors = neighbor_map[node["id"]]
        neighbor_labels = "\n".join(label[i] for i in neighbors)
        node["title"] += " Neighbors: \n" + neighbor_labels
        node["value"] = len(neighbor_map[node["id"]])

    nt.prep_notebook()
    print("\nUsers data stored in: ")
    nt.show(f'UserData/{user_name}.html')

def get_score(G, label, user_index, gcn_output, likes):
    user_embedding = gcn_output[user_index].detach().numpy()
    user_score = {
        other_user_index: float(1.0 / (1.0 + np.linalg.norm(user_embedding - gcn_output[other_user_index].detach().numpy())))
        for other_user_index in range(len(label)) if user_index != other_user_index
    }

    print("Compatibility Scores: ")
    for other_user_index, score in sorted(user_score.items(), key=lambda x: x[1], reverse=True):
        common_categories = likes[label[user_index]].intersection(likes[label[other_user_index]])
        if common_categories:
            print(f"{label[user_index]} --> {label[other_user_index]}: {score}\nCommon Categories: {', '.join(common_categories)}")

gcn_main()


3403
130
Compatibility Scores: 
Ashley1 --> Christina88743: 0.9995264270562726
Common Categories: Publisher, Video creator, Event Planner, Hospital, Digital creator, Dentist, Cause, Doctor
Ashley1 --> David39852: 0.9993274565749056
Common Categories: Event Planner, Digital creator, Photographer, Cause, Gaming, Doctor
Ashley1 --> Danielle14794: 0.9990321237957372
Common Categories: Publisher, Software company, Entertainer, Video creator, Event Planner, Hospital, Digital creator, Dentist, Cause, Gaming, Doctor
Ashley1 --> Julia92376: 0.9983422269760772
Common Categories: Entertainer, Video creator, Event Planner, Hospital, Photographer, Dentist
Ashley1 --> Ashley15061: 0.9981366610393672
Common Categories: Publisher, Software company, Video creator, Event Planner, Hospital, Digital creator, Cause, Gaming, Doctor
Ashley1 --> Shelly75535: 0.9972026735885681
Common Categories: Event Planner, Digital creator, Dentist, Cause, Gaming, Doctor
Ashley1 --> Stephen87302: 0.9966865662792785
Common 